In [ ]:
!pip install torch torchvision
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install faiss-cpu
!pip install faiss-gpu
!pip install faiss
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-um_caf8p
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-um_caf8p
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=e1a325df98b9ec69887630b9021ef255ef993409a5ef6dc886e637fbf2f4201a
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ks2tml5/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 78.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
ERROR: Could not find a version that satisfies the requirement faiss

In [ ]:
from google.colab import drive
import torch
import faiss
import numpy as np
from PIL import Image
from transformers import AlignProcessor, AlignModel
from torch.utils.data import DataLoader

# Step 1: Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
train_path       = '/content/drive/My Drive/Building_images_train/'
train_label_path = '/content/drive/My Drive/Building_images_train/train_labels.json'
# output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned/'
# output_dir = '/content/drive/My Drive/Image_text models/Clip_fine-tunned_conservative'
# output_dir = '/content/drive/My Drive/Image_text models/Clip_fine-tunned_conservative'


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dataset and DataLoader
image_dir = train_path # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch32/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

Epoch 29/30: 100%|██████████| 23/23 [00:12<00:00,  1.79it/s]


Epoch 29 Loss: 2.1621434170266856


Epoch 30/30: 100%|██████████| 23/23 [00:12<00:00,  1.80it/s]


Epoch 30 Loss: 2.1390307312426358


[]

In [ ]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
import os

# Load Fine-Tuned Model and Processor
model_path = output_dir
print(model_path)
model = CLIPModel.from_pretrained(model_path)
processor = CLIPProcessor.from_pretrained(model_path)

# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to Compute Similarity Scores
def get_similar_images(query, image_dir, top_k=5):
    """
    Args:
        query (str): The text query to search for.
        image_dir (str): Directory containing the images.
        top_k (int): Number of top similar images to return.

    Returns:
        List of tuples (image_name, score) and a list of corresponding images.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Prepare the text input
    text_inputs = processor(text=[query], return_tensors="pt", padding=True)
    text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

    # Get text embedding
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)  # Normalize

    # Compute similarity scores for all images
    image_scores = []
    images = {}

    for image_name in os.listdir(image_dir):
        if image_name.endswith(".json"):
            continue
        image_path = os.path.join(image_dir, image_name)
        image = Image.open(image_path).convert("RGB")

        # Process the image
        image_inputs = processor(images=image, return_tensors="pt")
        image_inputs = {k: v.to(device) for k, v in image_inputs.items()}

        # Get image embedding
        with torch.no_grad():
            image_features = model.get_image_features(**image_inputs)
            image_features /= image_features.norm(dim=-1, keepdim=True)  # Normalize

        # Compute cosine similarity
        similarity = torch.matmul(text_features, image_features.T).item()
        image_scores.append((image_name, similarity))
        images[image_name] = image

    # Sort by similarity score in descending order
    image_scores = sorted(image_scores, key=lambda x: x[1], reverse=True)

    # Get top_k results
    top_scores = image_scores[:top_k]
    top_images = [images[image_name] for image_name, _ in top_scores]
    return top_scores, top_images

# Display Results
def display_results(query, top_scores, top_images):
    """
    Displays the images with their similarity scores.

    Args:
        query (str): The text query.
        top_scores (list): List of tuples (image_name, score).
        top_images (list): List of PIL images.
    """
    print(f"Query: {query}")
    for i, (image_name, score) in enumerate(top_scores):
        print(f"{i+1}: {image_name} (Score: {score:.4f})")

    # Show images vertically with larger scales
    fig, axes = plt.subplots(len(top_images), 1, figsize=(5, 5 * len(top_images)))
    if len(top_images) == 1:
        axes = [axes]  # Ensure axes is iterable for a single image
    for ax, (image, (image_name, score)) in zip(axes, zip(top_images, top_scores)):
        ax.imshow(image)
        ax.set_title(f"{image_name}\nScore: {score:.4f}", fontsize=14)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

# Example Usage
query = "bath tub"
image_dir = train_path # Change to your image directory
# image_dir = '/content/drive/My Drive/Building_images_test_case/10130818'

top_scores, top_images = get_similar_images(query, image_dir, top_k=50)
display_results(query, top_scores, top_images)


# Normal But with batch size of 8

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dataset and DataLoader
image_dir = train_path # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch8/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

Epoch 29/30: 100%|██████████| 90/90 [00:15<00:00,  5.68it/s]


Epoch 29 Loss: 1.072802930408054


Epoch 30/30: 100%|██████████| 90/90 [00:15<00:00,  5.68it/s]


Epoch 30 Loss: 1.0546453360054229


[]

# Freezing 1layers




In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last few layers of the vision model
num_unfreeze_vision = 1  # Number of vision encoder layers to unfreeze
for block in model.vision_model.encoder.layers[-num_unfreeze_vision:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the last few layers of the text model
num_unfreeze_text = 1  # Number of text encoder layers to unfreeze
for block in model.text_model.encoder.layers[-num_unfreeze_text:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the projection layers
for param in model.visual_projection.parameters():
    param.requires_grad = True

for param in model.text_projection.parameters():
    param.requires_grad = True

# Dataset and DataLoader
image_dir = train_path  # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=5e-6
)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_freeze_1layer/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

Epoch 1/100: 100%|██████████| 90/90 [00:13<00:00,  6.48it/s]


Epoch 1 Loss: 1.689998409483168


Epoch 2/100: 100%|██████████| 90/90 [00:13<00:00,  6.70it/s]


Epoch 2 Loss: 1.4149100091722278


Epoch 3/100: 100%|██████████| 90/90 [00:13<00:00,  6.72it/s]


...

Epoch 99/100: 100%|██████████| 90/90 [00:13<00:00,  6.75it/s]


Epoch 99 Loss: 1.0464306357834074


Epoch 100/100: 100%|██████████| 90/90 [00:13<00:00,  6.74it/s]


Epoch 100 Loss: 1.0140284690592023


[]

# Freeze 2layers

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last few layers of the vision model
num_unfreeze_vision = 2  # Number of vision encoder layers to unfreeze
for block in model.vision_model.encoder.layers[-num_unfreeze_vision:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the last few layers of the text model
num_unfreeze_text = 2  # Number of text encoder layers to unfreeze
for block in model.text_model.encoder.layers[-num_unfreeze_text:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the projection layers
for param in model.visual_projection.parameters():
    param.requires_grad = True

for param in model.text_projection.parameters():
    param.requires_grad = True

# Dataset and DataLoader
image_dir = train_path  # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=5e-6
)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_freeze_2layer/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

Epoch 1/100: 100%|██████████| 90/90 [00:14<00:00,  6.38it/s]


Epoch 1 Loss: 1.6508085979355707


Epoch 2/100: 100%|██████████| 90/90 [00:13<00:00,  6.57it/s]


Epoch 2 Loss: 1.3420739541451137


Epoch 3/100: 100%|██████████| 90/90 [00:13<00:00,  6.61it/s]


...

Epoch 99/100: 100%|██████████| 90/90 [00:13<00:00,  6.54it/s]


Epoch 99 Loss: 1.0099311043818793


Epoch 100/100: 100%|██████████| 90/90 [00:13<00:00,  6.54it/s]


Epoch 100 Loss: 1.0118419922060438


[]

# Freezing 10layer

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last few layers of the vision model
num_unfreeze_vision = 10  # Number of vision encoder layers to unfreeze
for block in model.vision_model.encoder.layers[-num_unfreeze_vision:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the last few layers of the text model
num_unfreeze_text = 10  # Number of text encoder layers to unfreeze
for block in model.text_model.encoder.layers[-num_unfreeze_text:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the projection layers
for param in model.visual_projection.parameters():
    param.requires_grad = True

for param in model.text_projection.parameters():
    param.requires_grad = True

# Dataset and DataLoader
image_dir = train_path  # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=5e-6
)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_freeze_10layer/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

Epoch 1/100: 100%|██████████| 90/90 [00:16<00:00,  5.49it/s]


Epoch 1 Loss: 1.6042993783950805


Epoch 2/100: 100%|██████████| 90/90 [00:16<00:00,  5.62it/s]


Epoch 2 Loss: 1.2681684679455227


Epoch 3/100: 100%|██████████| 90/90 [00:15<00:00,  5.67it/s]


...

Epoch 99/100: 100%|██████████| 90/90 [00:15<00:00,  5.64it/s]


Epoch 99 Loss: 1.0105150471131006


Epoch 100/100: 100%|██████████| 90/90 [00:15<00:00,  5.63it/s]


Epoch 100 Loss: 1.0248105817370945


[]

# Info NCE Loss

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Define the InfoNCE Loss Function
def info_nce_loss(image_embeddings, text_embeddings, logit_scale):
    """
    Computes the InfoNCE loss for the CLIP model.

    Args:
        image_embeddings: Image embeddings tensor.
        text_embeddings: Text embeddings tensor.
        logit_scale: Logit scaling factor (usually obtained from the model).

    Returns:
        Scalar loss value.
    """
    # Normalize the embeddings
    image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
    text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)

    # Compute similarity logits
    logits = torch.matmul(image_embeddings, text_embeddings.T) * logit_scale

    # Ground truth labels (diagonal elements are positives)
    batch_size = image_embeddings.size(0)
    labels = torch.arange(batch_size).to(image_embeddings.device)

    # Cross-entropy loss
    loss_i2t = torch.nn.functional.cross_entropy(logits, labels)
    loss_t2i = torch.nn.functional.cross_entropy(logits.T, labels)
    loss = (loss_i2t + loss_t2i) / 2.0
    return loss

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dataset and DataLoader
image_dir = train_path  # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Forward pass
        outputs = model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        image_embeddings = outputs.image_embeds
        text_embeddings = outputs.text_embeds
        logit_scale = model.logit_scale.exp()

        # Compute InfoNCE loss
        loss = info_nce_loss(image_embeddings, text_embeddings, logit_scale)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_NCEloss/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)


Epoch 1/100: 100%|██████████| 90/90 [00:17<00:00,  5.24it/s]


Epoch 1 Loss: 1.5853077431519826


Epoch 2/100: 100%|██████████| 90/90 [00:16<00:00,  5.44it/s]


Epoch 2 Loss: 1.2576130277580686


Epoch 3/100: 100%|██████████| 90/90 [00:16<00:00,  5.43it/s]


...

Epoch 99/100: 100%|██████████| 90/90 [00:16<00:00,  5.39it/s]


Epoch 99 Loss: 1.0406276524066924


Epoch 100/100: 100%|██████████| 90/90 [00:16<00:00,  5.42it/s]


Epoch 100 Loss: 1.0291113393174278


[]

# Batch 1

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dataset and DataLoader
image_dir = train_path # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch1/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

# Batch 2

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dataset and DataLoader
image_dir = train_path # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch2/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

Epoch 1/50: 100%|██████████| 360/360 [00:33<00:00, 10.66it/s]


Epoch 1 Loss: 0.5422555976152782


Epoch 2/50: 100%|██████████| 360/360 [00:32<00:00, 10.99it/s]


Epoch 2 Loss: 0.36681859780605414


Epoch 3/50: 100%|██████████| 360/360 [00:32<00:00, 10.98it/s]


...

Epoch 49/50: 100%|██████████| 360/360 [00:32<00:00, 11.08it/s]


Epoch 49 Loss: 0.27804317695382047


Epoch 50/50: 100%|██████████| 360/360 [00:32<00:00, 11.03it/s]


Epoch 50 Loss: 0.23301027235432029


[]

# batch 16

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dataset and DataLoader
image_dir = train_path # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_batch16/'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

Epoch 1/30: 100%|██████████| 45/45 [00:13<00:00,  3.30it/s]


Epoch 1 Loss: 2.233604245715671


Epoch 2/30: 100%|██████████| 45/45 [00:13<00:00,  3.32it/s]


Epoch 2 Loss: 1.8275928576787313


Epoch 3/30: 100%|██████████| 45/45 [00:13<00:00,  3.30it/s]


...

Epoch 29/30: 100%|██████████| 45/45 [00:13<00:00,  3.29it/s]


Epoch 29 Loss: 1.5786115831798977


Epoch 30/30: 100%|██████████| 45/45 [00:13<00:00,  3.30it/s]


Epoch 30 Loss: 1.5666036817762587


[]

#test

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last few layers of the vision model
num_unfreeze_vision = 2  # Number of vision encoder layers to unfreeze
for block in model.vision_model.encoder.layers[-num_unfreeze_vision:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the last few layers of the text model
num_unfreeze_text = 2  # Number of text encoder layers to unfreeze
for block in model.text_model.encoder.layers[-num_unfreeze_text:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the projection layers
for param in model.visual_projection.parameters():
    param.requires_grad = True

for param in model.text_projection.parameters():
    param.requires_grad = True

# Dataset and DataLoader
image_dir = train_path  # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity
        logits_per_text = outputs.logits_per_text  # Text-to-image similarity

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss
        loss = (loss_fn(logits_per_image, targets) + loss_fn(logits_per_text, targets)) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Update scheduler
    avg_loss = total_loss / len(dataloader)
    scheduler.step(avg_loss)  # Step the scheduler based on average loss

    print(f"Epoch {epoch + 1} Loss: {avg_loss}")

# Save the Fine-tuned Model
output_dir = '/content/drive/My Drive/Image_text models/CLIP_fine-tunned_test/'
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)


Epoch 1/100: 100%|██████████| 45/45 [00:12<00:00,  3.55it/s]


Epoch 1 Loss: 2.3692666901482475


Epoch 2/100: 100%|██████████| 45/45 [00:12<00:00,  3.59it/s]


Epoch 2 Loss: 1.9242683675554064


Epoch 3/100: 100%|██████████| 45/45 [00:12<00:00,  3.67it/s]


...

Epoch 99/100: 100%|██████████| 45/45 [00:12<00:00,  3.65it/s]


Epoch 99 Loss: 1.5043695158428616


Epoch 100/100: 100%|██████████| 45/45 [00:12<00:00,  3.66it/s]


Epoch 100 Loss: 1.4822502970695495


[]

# Conservative loss

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor, CLIPModel
import os
import json
from PIL import Image
from tqdm import tqdm

# Dataset Class for Variable Label Lengths
class ImageTextDataset(Dataset):
    def __init__(self, image_dir, json_path, processor):
        """
        Args:
            image_dir (str): Path to the directory containing images.
            json_path (str): Path to the JSON file with image-label mappings.
            processor: CLIP processor for preprocessing.
        """
        self.image_dir = image_dir
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.samples = [
            (image_name, label)
            for image_name, labels in self.data.items()
            for label in labels  # Flatten to image-label pairs
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_name, label = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        return image, label, image_name

# Custom Collate Function
def collate_fn(batch):
    """
    Custom collate function to handle variable-length text inputs.

    Args:
        batch: List of tuples (image, label, image_name).

    Returns:
        Processed inputs, image_names, labels.
    """
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    image_names = [item[2] for item in batch]

    # Process images and texts separately
    inputs = processor(
        text=labels,
        images=images,
        return_tensors="pt",
        padding=True
    )
    return inputs, image_names, labels

# Load Pre-trained CLIP Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dataset and DataLoader
image_dir = train_path  # Replace with your image directory
json_path = train_label_path  # Replace with your label file path
dataset = ImageTextDataset(image_dir, json_path, processor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Training Setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)

# Fine-Tuning Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, _, _ = batch
        pixel_values = inputs["pixel_values"].to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get embeddings
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
        image_embeds = outputs.image_embeds  # Image embeddings
        text_embeds = outputs.text_embeds  # Text embeddings

        # Normalize embeddings
        image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
        text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

        # Compute similarity matrix
        logits_per_image = image_embeds @ text_embeds.t()
        logits_per_text = text_embeds @ image_embeds.t()

        # Target: Single correct pair per batch
        targets = torch.arange(len(logits_per_image)).to(device)

        # Compute loss (CLIP symmetric loss)
        loss_image = torch.nn.functional.cross_entropy(logits_per_image, targets)
        loss_text = torch.nn.functional.cross_entropy(logits_per_text, targets)
        loss = (loss_image + loss_text) / 2
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

output_dir = '/content/drive/My Drive/Image_text models/Clip_fine-tunned_conservative'

# Save the Fine-tuned Model
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)


Epoch 1/100: 100%|██████████| 45/45 [00:14<00:00,  3.02it/s]


Epoch 1 Loss: 2.61843310991923


Epoch 2/100: 100%|██████████| 45/45 [00:14<00:00,  3.15it/s]


Epoch 2 Loss: 2.391757493548923


Epoch 3/100: 100%|██████████| 45/45 [00:14<00:00,  3.13it/s]


...

Epoch 99/100: 100%|██████████| 45/45 [00:13<00:00,  3.26it/s]


Epoch 99 Loss: 2.241205750571357


Epoch 100/100: 100%|██████████| 45/45 [00:13<00:00,  3.25it/s]


Epoch 100 Loss: 2.241130797068278


[]